In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import os
import tensorflow as tf

# Cargar el modelo entrenado
model = tf.keras.models.load_model("asl_model.keras")
print("Modelo cargado.")

# Mapeo de etiquetas numéricas a letras
unique_labels = sorted(os.listdir("..\\ASL\\asl_dataset\\"))
num_to_label = {idx: label for idx, label in enumerate(unique_labels)}

# Inicialización de MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer puntos clave de una imagen
def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints).flatten()
    return None

# Capturar video en tiempo real
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Extraer puntos clave de la imagen actual
    keypoints = extract_keypoints(frame)
    if keypoints is not None:
        # Clasificar la letra
        keypoints = np.expand_dims(keypoints, axis=0)
        prediction = model.predict(keypoints)
        predicted_label = num_to_label[np.argmax(prediction)]

        # Mostrar la predicción en pantalla
        cv2.putText(frame, predicted_label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Mostrar la imagen
    cv2.imshow("ASL Interpreter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
